In [1]:
import csv
from pickle import dump, load
from tqdm import tqdm
v='07.12.23'
topN = 5

prost_res = {}
with open(f'{v}.0.05.prost.res.tsv','r') as f:
    for line in f:
        line = line.strip().split('\t')
        q = line[0]
        h = line[2]
        if q not in prost_res: prost_res[q] = []
        if len(prost_res[q]) < topN: prost_res[q].append(h)

In [2]:
with open(f'../pkl/union.db.all.go.{v}.pkl','rb') as f:
    union_fa_all_upd = load(f)
with open('../pkl/IA.dict.pkl','rb') as f:
    IAdict = load(f)

In [3]:
#filter out go terms with IA = 0
#merge the go terms from homologs, but dont mix the ranking.
with open(f'{v}.prost.e005.top{topN}.tsv','w') as f:
    for p in prost_res:
        goMap = {}
        for i,h in enumerate(prost_res[p]):
            score = 1 - i*(1/topN)
            for go in union_fa_all_upd[h]:
                if go not in IAdict: continue
                if IAdict[go] == 0: continue
                if go not in goMap: goMap[go] = score
        for go in goMap:
            f.write(f'{p}\t{go}\t{goMap[go]}\n')

In [4]:
!tar -czf 07.12.23.e005.top5.tar.gz 07.12.23.prost.e005.top5.tsv